# Intro

In this notebook, we simulate the field created by the TRIF electromagnet and save it as a data set. This data set is going to be used to test our calibration approach.

We model the electromagnet as three magnetic dipoles oriented in the same direction. The dipoles are located in the same plain at the same distance from the axis of symmetry, and their position vectors are at $120^0$ with respect to each other. We also assume that there can be a uniform external magnetic field.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
def mag(x):
    """Calculates magnitude of vector x"""
    return np.sqrt(np.dot(x,x))


def dipole_field(r, r0, m, a, B_uniform):
    """r - location of the field
       r0 - position of the dipole
       m - magnitude of the dipole
       a - dipole axis"""
    M = m*a   
    R = r-r0
    if np.dot(R,R) == 0:
        return np.array([0, 0, 0])
    return mu0*(3*R*(np.dot(M,R))/(mag(R)**5) - M/(mag(R)**3)) + B_uniform

In [5]:
mu0 = 10**-7# magnetic constant

m = 1e8 #magnetic dipole magnitude
#A = np.array([0,0.5,1]) #magnetic dipole axis
R1 = np.array([1, 0, 1]) #magnetic dipole1 position
R2 = np.array([-np.cos(np.pi/3), np.sin(np.pi/3), 1]) #magnetic dipole2 position
R3 = np.array([-np.cos(np.pi/3), -np.sin(np.pi/3), 1]) #magnetic dipole3 position

A1 = np.array([0, 0, 1]) #dipole 1 axis
A2 = np.array([0, 0, 1])# dipole 2 axis
A3 = np.array([0, 0, 1])# dipole 3 axis

r0 = np.array([0., 0., 0.])# sensor position


B_uniform = np.array([0.,0.,0.]) # uniform magnetic field

In [6]:
def total_field(r, M):
    B1 = dipole_field(r, R1, M[0], A1, B_uniform)
    B2 = dipole_field(r, R2, M[1], A2, B_uniform)
    B3 = dipole_field(r, R3, M[2], A3, B_uniform)
    return B1+B2+B3

In [8]:
N1 = 501 # number of training samples for each dipole
M1 = np.concatenate((np.linspace(-10, 10, N1), np.zeros((N1)), np.zeros(N1)))
M2 = np.concatenate((np.zeros((N1)), np.linspace(-10, 10, N1),  np.zeros(N1)))
M3 = np.concatenate((np.zeros((N1)), np.zeros(N1), np.linspace(-10, 10, N1)))

targets=np.stack((M1, M2, M3), axis=1)

In [10]:
N2 = 1000 # 

M = np.random.uniform(-10, 10, (N2, 3))

targets_train = np.concatenate((targets, M))

array([[ 39.39674625, -44.56902771,   0.29159534],
       [ -2.85143834, -40.70641122,  -9.39833774],
       [-23.4022467 ,  -1.2253115 ,  11.01270276],
       [-46.30480683,  -2.53196231,  20.76791632],
       [-39.01847508, -63.18613903,  -6.35479905],
       [ 23.02178685,  51.78354813,   2.06312332],
       [ 29.34431678,  55.87896934,   4.12428144],
       [  0.24141003,   6.92690776, -32.92589797],
       [ 56.36041573, -31.89728789,   2.36253699],
       [ 10.61469723, -36.93994558,   4.79567754],
       [  4.32039958, -42.75303842,  21.67992002],
       [ -3.64898075,  15.68275344,  -3.99450058],
       [ -3.29439309, -39.75004107,  27.81606159],
       [ 34.90960462, -19.28176982, -25.95254289],
       [ -5.31731177,   2.73877974,  43.27209466],
       [ 56.16557908,   5.88064175, -17.53235435],
       [-56.98800704, -20.86647678,  -9.0553554 ],
       [  4.49841839, -17.05539242,  44.47579374],
       [-61.23131563,  12.89918552,  -6.43465225],
       [ 62.77113018, -60.63384

In [ ]:
B = []

for M in targets_train:
    B.append(total_field(r0, M*m))
    
B_train = np.array(B)

Training_set = pd.DataFrame(np.concatenate((B_train, targets_train), axis = 1), columns = ["B_x", "B_y", "B_z", "M_1", "M_2", "M_3"])

# np.save("data\B_train.npy", B_train)
# np.save("data\M_train.npy", targets_train)

In [ ]:
N_test = 100

targets_test = np.random.uniform(-10, 10, (N_test, 3))
B = []

for M in targets_test:
    B.append(total_field(r0, M*m))
    
B_test = np.array(B)

# np.save("data\B_test.npy", B_test)
# np.save("data\M_test.npy", targets_test)
